<div><img style="float: right; width: 120px; vertical-align:middle" src="https://www.upm.es/sfs/Rectorado/Gabinete%20del%20Rector/Logos/EU_Informatica/ETSI%20SIST_INFORM_COLOR.png" alt="ETSISI logo" />

# A ver si nos hacemos ricos<a id="top"></a>

<i>Última actualización: 2024-03-07</small></i></div>
***

## Introducción

Vamos a explorar el comportamiento de una red recurrente en el problema de predicción bursátil. Se trata de un problema muy difícil de resolver por varias razones.

En primer lugar, el mercado de valores es un sistema **altamente complejo** y dinámico, en el que multitud de factores pueden influir en los precios de las acciones. Estos factores incluyen las condiciones económicas mundiales, los acontecimientos políticos, los resultados de las empresas y el sentimiento de los inversores, entre otros. Por lo tanto, es difícil captar y modelizar todas estas complejas relaciones utilizando una única arquitectura RNN.

En segundo lugar, el mercado de valores se caracteriza por **patrones no estacionarios y no lineales**, en los que las relaciones entre entradas y salidas pueden cambiar con el tiempo. Las RNN son particularmente adecuadas para modelizar datos de series temporales, pero pueden tener dificultades para captar patrones no lineales y no estacionarios, especialmente en horizontes temporales largos.

En tercer lugar, el mercado bursátil **está sujeto a acontecimientos impredecibles** que pueden tener un impacto significativo en los precios de las acciones. Por ejemplo, las noticias inesperadas o los cambios repentinos en la confianza del mercado pueden provocar cambios importantes y rápidos en los precios de las acciones que pueden ser difíciles de predecir utilizando únicamente datos históricos.

Por último, el mercado bursátil es un entorno altamente competitivo, en el que inversores y operadores utilizan diversas técnicas sofisticadas para obtener una ventaja en el mercado. Esto significa que cualquier modelo de predicción debe ser capaz de superar al mercado para ser útil, lo cual es una tarea difícil.

## Objetivos

Crearemos un modelo de regresión para intentar predecir el valor de cierre que tendrán las acciones de Google (<https://es.finance.yahoo.com/quote/goog/>).

## Bibliotecas y configuración

A continuación importaremos las bibliotecas que se utilizarán a lo largo del cuaderno.

In [ ]:
import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler

También configuraremos algunos parámetros para adaptar la presentación gráfica.

In [ ]:
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams.update({'figure.figsize': (20, 6),'figure.dpi': 64})

***

## Descarga y preprocesamiento de datos

Empezaremos cargando los datos del conjunto de datos de acciones de Google. Para ello tomaremos los datos históricos de la empresa de los últimos 5 años.

In [ ]:
CODE = 'GOOG'
BASE_URL = f'https://query1.finance.yahoo.com/v7/finance/download/{CODE}'
# Interval
today = datetime.datetime.now()
five_years_ago = today - datetime.timedelta(days=365*5)
# Timestamps (as integer) for the interval
period2 = int(today.timestamp())
period1 = int(five_years_ago.timestamp())
# Now get the dataframe
df = pd.read_csv(
    f'{BASE_URL}?period1={period1}&period2={period2}&interval=1d',
    index_col='Date',
    parse_dates=['Date'],
    dtype=np.float32
)
df.tail()

El conjunto de datos que hemos descargado guarda la información del día. Nuestro objetivo será predecir el valor de cierre del día a partir de los datos de la semana pasada, para saber qué tipo de inversión nos conviene.

Alimentaremos la red con datos de hace 10 días para ver si la memoria sobre cómo evolucionan los valores se traduce en una mejor predicción. Para ello, empezaremos creando 95 nuevas columnas con el contenido de las columnas en cada paso temporal consecutivo.

Nótese que primero debemos ordenar el conjunto en orden ascendente para asegurarnos de que tenemos los días correctamente colocados. En segundo lugar, la primera

In [ ]:
SEQUENCE_LEN = 10

df.sort_index(inplace=True)
columns = df.columns
for i in range(1, SEQUENCE_LEN):
    for column in columns:
        df[f'{column} t-{i}'] = df[column].shift(i)
df.drop(df.head(SEQUENCE_LEN - 1).index, inplace=True)
df.head()

Continuaremos creando una nueva columna llamada `Y` con los valores de la columna `Close/Last` de una semana más tarde. Al igual que antes, las últimas 7 filas del conjunto de datos no nos servirán de nada, ya que no podemos ver el futuro, por lo que las eliminaremos.

In [ ]:
DAYS_IN_FUTURE = 7

df['Y'] = df['Close'].shift(-DAYS_IN_FUTURE)
df.drop(df.tail(DAYS_IN_FUTURE).index, inplace=True)

# Plot the dataset 
df[[c for c in df.columns if c in ('Close', 'Y')]].tail(DAYS_IN_FUTURE + 1)

Vamos a extraer un conjunto de prueba para comprobar el comportamiento de nuestro modelo. Este estará compuesto por los últimos 100 días del _dataset_. Estos datos serán normalizados entre 0 y 1 con los normalizadores que hemos creado, y ya que estamos aprovecharemos y generaremos los datos de entrada y salida.

Una nota; Para las redes recurrentes se espera que la entrada sea tridimensional, pero nuestro conjunto de datos es bidimensional. Tenemos que transformarlo a tridimensional, manteniendo la última dimensión de tamaño 1.

In [ ]:
TEST_SIZE = 100

train_df, test_df = df.iloc[:-TEST_SIZE,:], df.iloc[-TEST_SIZE:,:]

Ahora normalizaremos los valores entre 0 y 1 para que el impacto de las entradas sea similar y no haya ninguna que destaque. Para ello haremos uso de la clase `MinMaxScaler`, que además de normalizar, mantiene el estado para que también podamos usarlo como mecanismo de desnormalización.

In [ ]:
input_scaler = MinMaxScaler(feature_range=(0, 1))
input_scaler.fit(train_df.iloc[:,:-1])
output_scaler = MinMaxScaler(feature_range=(0, 1))
output_scaler.fit(train_df.iloc[:,-1:]);

Nótese que al normalizar, hemos normalizado sólo con el conjunto de entrenamiento. La idea subyacente es que el conjunto de prueba debe ser desconocido no sólo al crear el modelo, sino también al trabajar con los datos.

Ahora, vamos a ver en una gráfica a qué valores se corresponden estos conjuntos.

In [ ]:
x_train, y_train = train_df.iloc[:,:-1], train_df.iloc[:,-1:]
x_test, y_test = test_df.iloc[:,:-1], test_df.iloc[:,-1:]

plt.plot(y_train, color='green', label='Train')
plt.plot(y_test, color='red', label='Test')
plt.title('GOOG Stock Price Datset')
plt.xlabel('Day')
plt.ylabel('GOOG Stock Price')
plt.show()

Finally, we will transform the data from the training and test sets.

In [ ]:
x_train = input_scaler.transform(x_train)
x_test = input_scaler.transform(x_test)
y_train = output_scaler.transform(y_train)

x_train = np.reshape(x_train, (
    x_train.shape[0],                 # Examples
    SEQUENCE_LEN,                     # Number of timesteps
    x_train.shape[1] // SEQUENCE_LEN  # Number of components each sequence element
))
x_test = np.reshape(x_test, (x_test.shape[0], SEQUENCE_LEN, x_test.shape[1] // SEQUENCE_LEN))

print(f'Dataset shape:      {df.shape}')
print(f'Training: X shape = {x_train.shape}; Y shape = {y_train.shape}')
print(f'Test:     X shape = {x_test.shape}; Y shape = {y_test.shape}')

## Implementando y entrenando nuestro modelo

Crearemos un modelo recurrente que, a partir de los datos de entrada (los datos bursátiles de un día determinado) sea capaz de inferir el valor de cierre del valor siete días después. Para ello, utilizaremos una unidad neuronal recurrente simple `SimpleRNN`.

En concreto, la red que crearemos estará compuesta por dos capas de unidades recurrentes de 128 parámetros con una capa de abandono después de cada una de ellas, de la siguiente manera:

`Input`->`SimpleRNN`->`Dropout`->`SimpleRnn`->`Dropout`->`Output`.

Recuerda que cuando las unidades recurrentes se apilan unas sobre otras debemos utilizar el parámetro `return_sequences` para que se conecten a la secuencia completa, de lo contrario dará error.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.SimpleRNN(128, return_sequences=True, input_shape=(10, 6)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.SimpleRNN(128),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
])

model.compile(loss='mse', optimizer='adam', metrics=[tf.keras.metrics.RootMeanSquaredError()])
model.summary()

Ahora entrenaremos el modelo, a ver qué tal se comporta.

In [ ]:
history = model.fit(x_train, y_train, epochs=100)

In [ ]:
plt.plot(history.history['root_mean_squared_error'])
plt.ylabel('rmse')
plt.xlabel('epoch')
plt.show()

No sólo no ha ido mal, ¡ha ido genial! Igual hasta nos hacemos ricos y todo...

## Evaluando el modelo

Finalmente vamos a predecir para el conjunto de test los valores de las acciones para luego compararlos con los valores de verdad.

Una nota: los datos del test vienen con un índice de fecha (es un `Series` de pandas) mientras que la predicción viene como un array de `NumPy`. Hay que conseguir que ambos índices coincidan.

In [ ]:
predicted = model.predict(x_test)
# Denormalize the output
predicted = output_scaler.inverse_transform(predicted)

plt.plot(y_test.to_numpy(), label = 'Real')
plt.plot(predicted, label = 'Predicted')
plt.title('GOOG Stock Price Prediction')
plt.xlabel('Day')
plt.ylabel('GOOG Stock Price')
plt.legend()
plt.show();

A la vista de los resultados, parece que está ocurriendo algo extraño... ¿Vamos a hacernos ricos o no? ¿Se te ocurre alguna explicación para esta curiosa predicción? ¿A alguien se le ocurre una alternativa?

## Conclusiones

Hemos visto cómo implementar redes neuronales recurrentes en un problema de series temporales puras. También hemos visto que no se comporta demasiado bien, pero no tiene por qué ser así (en este caso hay truco).

Las series temporales mantienen correlaciones entre elementos consecutivos, y esas relaciones son las que aprenden las redes. Cuanto mayor sea la correlación (absoluta) entre elementos, más probabilidades tendrá la red de adivinarla. Sin embargo, las sucesiones de elementos valorados aleatoriamente (que no tienen nada que ver entre sí) no son problemas que puedan resolverse con este tipo de redes. Ni siquiera son series temporales por definición.

***

<div><img style="float: right; width: 120px; vertical-align:top" src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" alt="Creative Commons by-nc-sa logo" />

[Volver al inicio](#top)

</div>